In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
path = "/Users/gumuhan/Documents/Courses/SYDE675/RecSys/dating/SpeedDatingData.csv"
dt = pd.read_csv(path, encoding = "gbk")
dt

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8373,552,22.0,1,44,2,21,22,14,10.0,5,...,8.0,5.0,7.0,6.0,7.0,9.0,5.0,9.0,5.0,6.0
8374,552,22.0,1,44,2,21,22,13,10.0,4,...,8.0,5.0,7.0,6.0,7.0,9.0,5.0,9.0,5.0,6.0
8375,552,22.0,1,44,2,21,22,19,10.0,10,...,8.0,5.0,7.0,6.0,7.0,9.0,5.0,9.0,5.0,6.0
8376,552,22.0,1,44,2,21,22,3,10.0,16,...,8.0,5.0,7.0,6.0,7.0,9.0,5.0,9.0,5.0,6.0


In [6]:
dt = dt.astype({"iid":"int","pid":"int"})

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

## Pure Elo

In [30]:
def elo(init_score=50):
    whole_elo_score = []
    wave_list = []
    iid_list = []
    
    for w in dt.wave.unique():
           
        # get sub df
        sub = dt[dt.wave == w][['iid', 'pid', 'dec_o', 'order']].dropna()
        
        sub_iid_list = sub.iid.unique()
        wave_list += [w]*len(sub.iid.unique())
        
        # start elo of this wave
        K = (100-init_score)/len(sub.iid.unique())
        elo_score = [init_score]*len(sub.iid.unique())
        min_iid = min(sub.iid)
        for order in sub.order.unique():
            sub_od = sub[sub.order == order]
            for idx, row in sub_od.iterrows():
                try:
                    sub_iid = int(row.iid)
                    sub_pid = int(row.pid)
                    i_score = elo_score[sub_iid - min_iid]
                    p_score = elo_score[sub_pid - min_iid]
                    elo_score[sub_iid - min_iid] += K*(row.dec_o - i_score/(i_score+p_score))
                except:
                    continue
        
        # sort
        sort_indices = np.argsort(elo_score)[::-1]
        elo_score = [elo_score[i] for i in sort_indices]
        sub_iid_list = [sub_iid_list[i] for i in sort_indices]
        
        iid_list += sub_iid_list
        whole_elo_score += elo_score
        
    return wave_list, iid_list, whole_elo_score

wave_list, iid_list, pure_elo = elo()

In [31]:
d_elo = pd.DataFrame(zip(wave_list, iid_list, pure_elo),
                     columns = ['wave', 'iid', 'elo_score'])
d_elo

,wave,iid,elo_score
0,1,19,59.746477
1,1,20,59.432085
2,1,14,57.329997
3,1,8,57.304343
4,1,17,57.267572
...,...,...,...
546,21,529,41.224796
547,21,527,41.182526
548,21,530,41.175093
549,21,523,40.288495


In [34]:
i_df = pd.read_csv('individual_all.csv')
d_elo_info = d_elo.merge(i_df.drop(columns = ['wave']), on='iid')
d_elo_info
d_elo_info.to_csv('elo_info.csv')

,wave,iid,elo_score,#dec_0,percent_dec_0,av_like_o,gender,idg,condtn,round,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,19,59.746477,9,0.900000,7.600000,1,18,1,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,20,59.432085,9,0.900000,8.300000,1,20,1,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,14,57.329997,8,0.800000,7.700000,1,8,1,10,...,9.0,9.0,9.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN
3,1,8,57.304343,8,0.800000,7.700000,0,15,1,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,17,57.267572,8,0.800000,6.900000,1,14,1,10,...,7.0,8.0,8.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,21,529,41.224796,3,0.136364,5.476190,0,41,2,22,...,6.0,10.0,7.0,8.0,7.0,6.0,9.0,7.0,8.0,8.0
547,21,527,41.182526,3,0.136364,5.000000,0,37,2,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
548,21,530,41.175093,3,0.136364,5.000000,0,43,2,22,...,6.0,9.0,5.0,7.0,5.0,3.0,8.0,8.0,5.0,5.0
549,21,523,40.288495,2,0.090909,5.454545,0,29,2,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## LR to Test

In [39]:
import statsmodels.api as sm
Y = d_elo_info.elo_score
X = d_elo_info[['#dec_0', 'percent_dec_0', 'av_like_o']]
#X
mod = sm.OLS(Y, X)
fii = mod.fit()
p_values = fii.summary2()#.tables[1]['P>|t|']
p_values
fii.summary2().tables[1]['P>|t|']

<class 'statsmodels.iolib.summary2.Summary'>
"""
                        Results: Ordinary least squares
===============================================================================
Model:                  OLS              Adj. R-squared (uncentered): 0.991    
Dependent Variable:     elo_score        AIC:                         3253.8716
Date:                   2022-04-27 14:16 BIC:                         3266.8068
No. Observations:       551              Log-Likelihood:              -1623.9  
Df Model:               3                F-statistic:                 2.007e+04
Df Residuals:           548              Prob (F-statistic):          0.00     
R-squared (uncentered): 0.991            Scale:                       21.372   
-----------------------------------------------------------------------------------
                  Coef.      Std.Err.        t        P>|t|       [0.025     0.975]
-----------------------------------------------------------------------------------
#dec_0            0.1392       0.0806      1.7273     0.0847     -0.0191     0.2975
percent_dec_0     2.5049       1.5075      1.6617     0.0972     -0.4562     5.4660
av_like_o         7.4347       0.0795     93.5154     0.0000      7.2785     7.5908
-------------------------------------------------------------------------------
Omnibus:                 120.179           Durbin-Watson:              1.578   
Prob(Omnibus):           0.000             Jarque-Bera (JB):           1015.009
Skew:                    0.692             Prob(JB):                   0.000   
Kurtosis:                9.504             Condition No.:              73      
===============================================================================

"""

#dec_0           0.084675
percent_dec_0    0.097154
av_like_o        0.000000
Name: P>|t|, dtype: float64

## Modified Elo by Combining PageRank & Iterative Elo

In [68]:
s = list(d_elo_info.columns).index('sports')
b = list(d_elo_info.columns).index('expnum')
this_d = d_elo_info[['elo_score']+list(d_elo_info.columns[s:b+1])].dropna()

Y = this_d.elo_score
X = this_d.drop(columns=['elo_score'])

#X
mod = sm.OLS(Y, X)
fii = mod.fit()
p_values = fii.summary2()#.tables[1]['P>|t|']
p_values
fii.summary2().tables[1]['P>|t|']

<class 'statsmodels.iolib.summary2.Summary'>
"""
                       Results: Ordinary least squares
==============================================================================
Model:                  OLS              Adj. R-squared (uncentered): 0.975   
Dependent Variable:     elo_score        AIC:                         889.1995
Date:                   2022-04-27 15:13 BIC:                         943.0888
No. Observations:       126              Log-Likelihood:              -425.60 
Df Model:               19               F-statistic:                 262.1   
Df Residuals:           107              Prob (F-statistic):          1.32e-80
R-squared (uncentered): 0.979            Scale:                       59.212  
----------------------------------------------------------------------------------
                 Coef.      Std.Err.        t        P>|t|       [0.025     0.975]
----------------------------------------------------------------------------------
sports           0.6046       0.3876      1.5598     0.1218     -0.1638     1.3730
tvsports        -0.2970       0.3246     -0.9151     0.3622     -0.9405     0.3464
exercise         0.6077       0.3527      1.7230     0.0878     -0.0915     1.3068
dining           1.2776       0.5082      2.5141     0.0134      0.2702     2.2850
museums         -0.0540       0.7692     -0.0702     0.9441     -1.5790     1.4709
art             -0.1567       0.6868     -0.2282     0.8200     -1.5182     1.2048
hiking           0.0068       0.3264      0.0208     0.9834     -0.6402     0.6538
gaming           0.0793       0.3178      0.2495     0.8035     -0.5507     0.7093
clubbing         1.1929       0.3815      3.1269     0.0023      0.4366     1.9492
reading          0.9554       0.4035      2.3677     0.0197      0.1555     1.7553
tv               0.5239       0.3441      1.5222     0.1309     -0.1584     1.2061
theater         -0.2542       0.4292     -0.5923     0.5549     -1.1050     0.5966
movies           1.6349       0.5977      2.7352     0.0073      0.4500     2.8198
concerts         0.0456       0.5159      0.0883     0.9298     -0.9771     1.0682
music            0.5391       0.5416      0.9955     0.3218     -0.5345     1.6128
shopping        -0.4788       0.3736     -1.2817     0.2027     -1.2195     0.2618
yoga             0.1669       0.2982      0.5596     0.5769     -0.4243     0.7580
exphappy         0.3474       0.4637      0.7492     0.4554     -0.5718     1.2666
expnum          -0.0673       0.1556     -0.4326     0.6662     -0.3759     0.2412
------------------------------------------------------------------------------
Omnibus:                  3.129             Durbin-Watson:               1.242
Prob(Omnibus):            0.209             Jarque-Bera (JB):            2.051
Skew:                     -0.086            Prob(JB):                    0.359
Kurtosis:                 2.399             Condition No.:               40   
==============================================================================

"""

sports      0.121760
tvsports    0.362213
exercise    0.087778
dining      0.013424
museums     0.944132
art         0.819962
hiking      0.983425
gaming      0.803455
clubbing    0.002275
reading     0.019693
tv          0.130900
theater     0.554920
movies      0.007300
concerts    0.929767
music       0.321751
shopping    0.202735
yoga        0.576907
exphappy    0.455381
expnum      0.666210
Name: P>|t|, dtype: float64

In [125]:
def m_elo(init_score=50, k=3):
    d_m_elo = pd.DataFrame(columns = ['wave', 'iid', 'm_elo_score'])
    d_all = pd.DataFrame()
    initial_weight = []
    wave_list = []
    iid_list = []
    
    for w in dt.wave.unique():      
        # get sub df
        sub = dt[dt.wave == w][['iid', 'pid', 'dec_o', 'order']].dropna()
        
        sub_iid_list = sub.iid.unique()
        sub_wave = [w]*len(sub.iid.unique())
        
        wave_list += sub_wave
        iid_list += list(sub_iid_list)
        # start elo of this wave
        K = (100-init_score)/len(sub.iid.unique())
        elo_score = [] # elo score of this wave
        
        
        # modified version -- improve a little bit to overcome cold-start
        # set initial weights of Elo Algorithm
        if w == 1:
            elo_score = [init_score]*len(sub.iid.unique())
        else:
            # measure similarity  ----------------------------------------  # measure similarity  
            fts = ['dining', 'clubbing', 'reading', 'movies']
            # iterate all iid in this wave
            p_list = list(d_all.iid.unique()) 
            #print(p_list)
            for i in sub_iid_list:  
                #print(i)
                dis_list = []
                for p in p_list:
                    dis = 0 # between i, p
                    for ft in fts:
                        dif = i_df[ft][list(i_df['iid']).index(i)] - d_all[ft][list(d_all['iid']).index(p)]
                        dis += dif**2
                    dis_list.append(dis)
                #print(dis_list)
                sort_idx = np.argsort(dis_list)[:k]
                p_list_sort = [p_list[i] for i in sort_idx]
                s_list = []
                #print(sort_idx)
                for p in p_list_sort:
                    #print(list(d_all['iid']).index(p))
                    #print(d_all['m_elo_score'])
                    to_add = d_all['m_elo_score'][list(d_all['iid']).index(p)]
                    #print(to_add)
                    
                    s_list.append(to_add)
                    #print(s_list)
                elo_score.append(sum(s_list)/k)
                #print(elo_score)
        initial_weight += elo_score
            
        min_iid = min(sub.iid)
        for order in sub.order.unique():
            sub_od = sub[sub.order == order]
            for idx, row in sub_od.iterrows():
                try:
                    sub_iid = int(row.iid)
                    sub_pid = int(row.pid)
                    i_score = elo_score[sub_iid - min_iid]
                    p_score = elo_score[sub_pid - min_iid]
                    elo_score[sub_iid - min_iid] += K*(row.dec_o - i_score/(i_score+p_score))
                except:
                    continue
        
        # sort
        sort_indices = np.argsort(elo_score)[::-1]
        elo_score = [elo_score[i] for i in sort_indices]
        sub_iid_list = [sub_iid_list[i] for i in sort_indices]
        
        d_m_elo = pd.concat([d_m_elo, 
                             pd.DataFrame({'wave': sub_wave,
                                          'iid': sub_iid_list,
                                          'm_elo_score': elo_score})],
                             ignore_index=True)
        d_all = d_m_elo.merge(i_df.drop(columns = ['wave']), on='iid')
        #print(d_all)
    #print(len(wave_list), len(iid_list), len(initial_weight))
    d_w = pd.DataFrame({'wave': wave_list,
                        'iid': iid_list,
                        'm_initial_weight': initial_weight,
                        'old_weight': [50]*len(iid_list)})
    return d_m_elo, d_w
    #return wave_list, iid_list, whole_elo_score

# wave_list, iid_list, m_elo = m_elo()

# d_m_elo = pd.DataFrame(zip(wave_list, iid_list, m_elo),
#                      columns = ['wave', 'iid', 'm_elo_score'])
d_m_elo, d_w = m_elo()
d_m_elo_info = d_m_elo.merge(i_df.drop(columns = ['wave']), on='iid')
d_m_elo_info
d_m_elo_info.to_csv('m_elo_info.csv', index=False)
d_w.to_csv('m_elo_comparison.csv', index=False)
d_w

,wave,iid,m_elo_score,#dec_0,percent_dec_0,av_like_o,gender,idg,condtn,round,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,19,59.746477,9,0.900000,7.600000,1,18,1,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,20,59.432085,9,0.900000,8.300000,1,20,1,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,14,57.329997,8,0.800000,7.700000,1,8,1,10,...,9.0,9.0,9.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN
3,1,8,57.304343,8,0.800000,7.700000,0,15,1,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,17,57.267572,8,0.800000,6.900000,1,14,1,10,...,7.0,8.0,8.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,21,517,31.016378,6,0.272727,5.294118,0,17,2,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
547,21,548,30.139672,10,0.454545,5.428571,1,36,2,22,...,8.0,9.0,9.0,9.0,9.0,8.0,9.0,9.0,9.0,7.0
548,21,527,27.331582,3,0.136364,5.000000,0,37,2,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
549,21,514,24.806177,4,0.181818,5.666667,0,11,2,22,...,9.0,10.0,10.0,9.0,7.0,10.0,10.0,10.0,10.0,8.0


,wave,iid,m_initial_weight,old_weight
0,1,1,50.000000,50
1,1,2,50.000000,50
2,1,3,50.000000,50
3,1,4,50.000000,50
4,1,5,50.000000,50
...,...,...,...,...
546,21,548,29.325313,50
547,21,549,52.065233,50
548,21,550,48.687929,50
549,21,551,49.219371,50


In [126]:
d_w_info = d_m_elo_info[d_m_elo_info.columns[:6]].merge(d_w, on='iid')
d_w_info
d_w_info.to_csv('d_w_info.csv', index=False)

,wave_x,iid,m_elo_score,#dec_0,percent_dec_0,av_like_o,wave_y,m_initial_weight,old_weight
0,1,19,59.746477,9,0.900000,7.600000,1,50.000000,50
1,1,20,59.432085,9,0.900000,8.300000,1,50.000000,50
2,1,14,57.329997,8,0.800000,7.700000,1,50.000000,50
3,1,8,57.304343,8,0.800000,7.700000,1,50.000000,50
4,1,17,57.267572,8,0.800000,6.900000,1,50.000000,50
...,...,...,...,...,...,...,...,...,...
546,21,517,31.016378,6,0.272727,5.294118,21,34.957112,50
547,21,548,30.139672,10,0.454545,5.428571,21,29.325313,50
548,21,527,27.331582,3,0.136364,5.000000,21,34.583472,50
549,21,514,24.806177,4,0.181818,5.666667,21,29.712153,50


In [119]:
d_m_elo_info.columns[:6]

Index(['wave', 'iid', 'elo_score', '#dec_0', 'percent_dec_0', 'av_like_o'], dtype='object')

In [112]:
d_m_elo_info

,wave,iid,elo_score,#dec_0,percent_dec_0,av_like_o,gender,idg,condtn,round,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,19,59.746477,9,0.900000,7.600000,1,18,1,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,20,59.432085,9,0.900000,8.300000,1,20,1,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,14,57.329997,8,0.800000,7.700000,1,8,1,10,...,9.0,9.0,9.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN
3,1,8,57.304343,8,0.800000,7.700000,0,15,1,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,17,57.267572,8,0.800000,6.900000,1,14,1,10,...,7.0,8.0,8.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,21,529,41.224796,3,0.136364,5.476190,0,41,2,22,...,6.0,10.0,7.0,8.0,7.0,6.0,9.0,7.0,8.0,8.0
547,21,527,41.182526,3,0.136364,5.000000,0,37,2,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
548,21,530,41.175093,3,0.136364,5.000000,0,43,2,22,...,6.0,9.0,5.0,7.0,5.0,3.0,8.0,8.0,5.0,5.0
549,21,523,40.288495,2,0.090909,5.454545,0,29,2,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
list(i_df['iid']).index(1)

0

In [116]:
[2]*len([1,1,1])

[2, 2, 2]

In [106]:
i_df['dining'].mean()
i_df['reading'].mean()
i_df['clubbing'].mean()
i_df['movies'].mean()

7.775735294117647

7.647058823529412

5.751838235294118

7.898897058823529

In [107]:
# 9 8 6 25/3 
9
23/3
6
25/3 

9

7.666666666666667

6

8.333333333333334

## M_Elo Consine Similarity 

In [121]:
# sample consine similarity 
from scipy import spatial

dataSetI = [3, 45, 7, 2]
dataSetII = [2, 54, 13, 15]
result = 1 - spatial.distance.cosine(dataSetI, dataSetII)
result

0.97228425171235

In [122]:
from numpy.linalg import norm
 
# define two lists or array
A = np.array([2,1,2,3,2,9])
B = np.array([3,4,2,4,5,5])
 
print("A:", A)
print("B:", B)
 
# compute cosine similarity
cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

A: [2 1 2 3 2 9]
B: [3 4 2 4 5 5]
Cosine Similarity: 0.8188504723485274


In [127]:
def m_elo_consine(init_score=50, k=3):
    d_m_elo = pd.DataFrame(columns = ['wave', 'iid', 'm_elo_score'])
    d_all = pd.DataFrame()
    initial_weight = []
    wave_list = []
    iid_list = []
    
    for w in dt.wave.unique():      
        # get sub df
        sub = dt[dt.wave == w][['iid', 'pid', 'dec_o', 'order']].dropna()
        
        sub_iid_list = sub.iid.unique()
        sub_wave = [w]*len(sub.iid.unique())
        
        wave_list += sub_wave
        iid_list += list(sub_iid_list)
        # start elo of this wave
        K = (100-init_score)/len(sub.iid.unique())
        elo_score = [] # elo score of this wave
        
        
        # modified version -- improve a little bit to overcome cold-start
        # set initial weights of Elo Algorithm.    ------------------------------------- Consine Similarity
        if w == 1:
            elo_score = [init_score]*len(sub.iid.unique())
        else:
            # measure similarity  ----------------------------------------  # measure similarity  
            fts = ['dining', 'clubbing', 'reading', 'movies']
            # iterate all iid in this wave
            p_list = list(d_all.iid.unique()) 
            #print(p_list)
            for i in sub_iid_list:  
                #print(i)
                pos_i = list(i_df['iid']).index(i)
                vec_i = np.array([i_df[ft][pos_i] for ft in fts])
                
                dis_list = []
                for p in p_list:
                    pos_p = list(d_all['iid']).index(p)
                    vec_p = np.array([d_all[ft][pos_p] for ft in fts])
                    dis = np.dot(vec_i, vec_p)/(norm(vec_i)*norm(vec_p))
                    
#                     dis = 0 # between i, p
#                     for ft in fts:
#                         dif = i_df[ft][list(i_df['iid']).index(i)] - d_all[ft][list(d_all['iid']).index(p)]
#                         dis += dif**2

                    dis_list.append(dis)
                #print(dis_list)
                sort_idx = np.argsort(dis_list)[:k]
                p_list_sort = [p_list[i] for i in sort_idx]
                s_list = []
                #print(sort_idx)
                for p in p_list_sort:
                    #print(list(d_all['iid']).index(p))
                    #print(d_all['m_elo_score'])
                    to_add = d_all['m_elo_score'][list(d_all['iid']).index(p)]
                    #print(to_add)
                    
                    s_list.append(to_add)
                    #print(s_list)
                elo_score.append(sum(s_list)/k)
                #print(elo_score)
        initial_weight += elo_score
            
        min_iid = min(sub.iid)
        for order in sub.order.unique():
            sub_od = sub[sub.order == order]
            for idx, row in sub_od.iterrows():
                try:
                    sub_iid = int(row.iid)
                    sub_pid = int(row.pid)
                    i_score = elo_score[sub_iid - min_iid]
                    p_score = elo_score[sub_pid - min_iid]
                    elo_score[sub_iid - min_iid] += K*(row.dec_o - i_score/(i_score+p_score))
                except:
                    continue
        
        # sort
        sort_indices = np.argsort(elo_score)[::-1]
        elo_score = [elo_score[i] for i in sort_indices]
        sub_iid_list = [sub_iid_list[i] for i in sort_indices]
        
        d_m_elo = pd.concat([d_m_elo, 
                             pd.DataFrame({'wave': sub_wave,
                                          'iid': sub_iid_list,
                                          'm_elo_score': elo_score})],
                             ignore_index=True)
        d_all = d_m_elo.merge(i_df.drop(columns = ['wave']), on='iid')
        #print(d_all)
    #print(len(wave_list), len(iid_list), len(initial_weight))
    d_w = pd.DataFrame({'wave': wave_list,
                        'iid': iid_list,
                        'mc_initial_weight': initial_weight,
                        'old_weight': [50]*len(iid_list)})
    return d_m_elo, d_w
    #return wave_list, iid_list, whole_elo_score

# wave_list, iid_list, m_elo = m_elo()

# d_m_elo = pd.DataFrame(zip(wave_list, iid_list, m_elo),
#                      columns = ['wave', 'iid', 'm_elo_score'])
d_m_elo_c, d_w_c = m_elo_consine()
d_m_elo_c_info = d_m_elo_c.merge(i_df.drop(columns = ['wave']), on='iid')
d_m_elo_c_info
d_m_elo_c_info.to_csv('m_elo_c_info.csv', index=False)
d_w_c.to_csv('m_elo_c_comparison.csv', index=False)
d_w_c

,wave,iid,m_elo_score,#dec_0,percent_dec_0,av_like_o,gender,idg,condtn,round,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,19,59.746477,9,0.900000,7.600000,1,18,1,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,20,59.432085,9,0.900000,8.300000,1,20,1,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,14,57.329997,8,0.800000,7.700000,1,8,1,10,...,9.0,9.0,9.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN
3,1,8,57.304343,8,0.800000,7.700000,0,15,1,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,17,57.267572,8,0.800000,6.900000,1,14,1,10,...,7.0,8.0,8.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,21,523,29.823744,2,0.090909,5.454545,0,29,2,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
547,21,527,29.220988,3,0.136364,5.000000,0,37,2,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
548,21,525,28.146760,0,0.000000,4.619048,0,33,2,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
549,21,529,28.122148,3,0.136364,5.476190,0,41,2,22,...,6.0,10.0,7.0,8.0,7.0,6.0,9.0,7.0,8.0,8.0


,wave,iid,mc_initial_weight,old_weight
0,1,1,50.000000,50
1,1,2,50.000000,50
2,1,3,50.000000,50
3,1,4,50.000000,50
4,1,5,50.000000,50
...,...,...,...,...
546,21,548,39.927408,50
547,21,549,39.348909,50
548,21,550,39.348909,50
549,21,551,39.465743,50


In [128]:
d_w_c_info = d_m_elo_c_info[d_m_elo_c_info.columns[:6]].merge(d_w, on='iid')
d_w_c_info
d_w_c_info.to_csv('d_w_c_info.csv', index=False)

,wave_x,iid,m_elo_score,#dec_0,percent_dec_0,av_like_o,wave_y,m_initial_weight,old_weight
0,1,19,59.746477,9,0.900000,7.600000,1,50.000000,50
1,1,20,59.432085,9,0.900000,8.300000,1,50.000000,50
2,1,14,57.329997,8,0.800000,7.700000,1,50.000000,50
3,1,8,57.304343,8,0.800000,7.700000,1,50.000000,50
4,1,17,57.267572,8,0.800000,6.900000,1,50.000000,50
...,...,...,...,...,...,...,...,...,...
546,21,523,29.823744,2,0.090909,5.454545,21,46.686697,50
547,21,527,29.220988,3,0.136364,5.000000,21,34.583472,50
548,21,525,28.146760,0,0.000000,4.619048,21,28.008353,50
549,21,529,28.122148,3,0.136364,5.476190,21,40.241971,50
